There i parse telegram chat history 

In [16]:
!pip install lxml

     ---------------------------------------- 3.6/3.6 MB 1.7 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\royta\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [21]:
import os
import shutil
from lxml import html
from contextlib import suppress

from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1

In [16]:
def get_name(body: html.HtmlElement) -> str|None:
    with suppress(Exception):
        name = body.find_class('from_name')[0].text
        name = name.replace('\n', '').strip()
        return name

In [17]:
def get_photo(body: html.HtmlElement) -> str|None:
    with suppress(Exception):
        div = body.find_class('media_wrap')[0]
        a = div.getchildren()[0]
        photo = a.attrib['href']
        return photo

In [18]:
def get_text(body: html.HtmlElement, text="") -> str|None:
    with suppress(Exception):
        div = body.find_class('text')[0]
        for t in div.itertext():
            text+= t
            
        text = text.replace('\n', '', 2).strip()
        return text

In [19]:
messages = []

for catalog in os.listdir('tg_data'):
    
    for file in os.listdir('tg_data/' + catalog):
        if '.html' in file:

            with open(f"tg_data/{catalog}/{file}", "r", encoding='utf-8') as f:
                _html = f.read()

            tree = html.fromstring(_html)
            history = tree.xpath("/html/body/div/div[2]/div")[0]
            history = history.find_class('message default clearfix')

            for message in history:
                body = message.find_class('body')[0]
                body = message.find_class('body')[0]
                    
                photo = get_photo(body) 
                photo = f"{catalog}/{photo}" if photo else None
                messages+= [{
                    'name': get_name(body),
                    'text': get_text(body),
                    'photo': photo,
                }]

In [22]:
mtcnn = MTCNN()
model1 = InceptionResnetV1(pretrained='vggface2').eval()
model2 = InceptionResnetV1(pretrained='casia-webface').eval()

100%|████████████████████████████████████████████████████████████████████████████████| 111M/111M [02:53<00:00, 667kB/s]


In [7]:
for current, previous in zip(messages[1:], messages[:-1]):
    if previous['photo']:
        photo_path = 'tg_data/' + previous['photo']
        photo_name = previous['photo'].split('/')[-1]
        
        if current['text'] in ['💌', '❤️', '👍']:
            saved_path = 'data/target/' + photo_name

        elif current['text'] in ['👎']:
            saved_path = 'data/opposite/' + photo_name
        
        else:
            continue
            
        
        while os.path.exists(saved_path):
            path, e = saved_path.split('.')
            path += ' i'
            saved_path = f"{path}.{e}"
            
        img = Image.open(photo_path)
        img_cropped = mtcnn(img, save_path=saved_path)

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [9]:
DATASET_PATH = "data"

In [8]:
for file in os.listdir('markup/test_target'):
    photo_path = 'markup/test_target/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/test_target/" + file
    
    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)
    
    
for file in os.listdir('markup/test_opposite'):
    photo_path = 'markup/test_opposite/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/test_opposite/" + file

    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)
    
    
for file in os.listdir('markup/target'):
    photo_path = 'markup/target/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/markup_target/" + file
    
    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)
    
    
for file in os.listdir('markup/opposite'):
    photo_path = 'markup/opposite/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/markup_opposite/" + file
    
    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Manually clear the dataset